In [146]:
import pandas as pd
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
from nltk.stem import PorterStemmer
import matplotlib.pyplot as plt
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from scipy.stats import spearmanr, pearsonr
from sklearn.linear_model import LinearRegression
import numpy as np
from sklearn.decomposition import NMF, LatentDirichletAllocation

In [147]:
trump_speech_df = pd.read_csv('C:/JK/Masters Studies/Fall 2017/KDD/Project 5/TrumpSpeech/MrTrumpSpeeches.csv',error_bad_lines=False, warn_bad_lines=False,header=0, delimiter="\~", quoting=3, engine='python')
# trump_speech_df.head()
def remove_nextline(text):
#     text = (re.sub("\n"," ",text).lower())
    return re.sub("[^a-zA-Z]"," ", text).lower()

trump_speech_df['subtitle1'] = trump_speech_df['subtitles'].apply(remove_nextline)

trump_speech_df['tokenized_subtitle'] = trump_speech_df['subtitle1'].apply(word_tokenize)
stopset = stopwords.words('english') + list(string.punctuation)
trump_speech_df['subtitle_stop']=trump_speech_df['tokenized_subtitle'] .apply(lambda x: [item for item in x if item not in stopset])
trump_speech_df.head()

,id,playlist,upload_date,title,view_count,average_rating,like_count,dislike_count,subtitles,subtitle1,tokenized_subtitle,subtitle_stop
0,-2WTNSujhjk,Donald Trump Speeches & Events,20160220,Live Stream: Donald Trump Victory Rally in Spa...,4057.0,4.259259,44.0,10.0,presidents of the United States mr. go tr...,presidents of the united states mr go tr...,"[presidents, of, the, united, states, mr, go, ...","[presidents, united, states, mr, go, trapp, fa..."
1,-64nfy6i58w,Donald Trump Speeches & Events,20161107,LAST RALLY: Donald Trump FINAL CAMPAIGN Rally ...,47276.0,4.358025,952.0,182.0,it's now officially Tuesday November a di...,it s now officially tuesday november a di...,"[it, s, now, officially, tuesday, november, a,...","[officially, tuesday, november, ever, think, h..."
2,-7Sp31hTxkU,Donald Trump Speeches & Events,20160423,"FULL SPEECH: Donald Trump Rally in Bridgeport,...",19966.0,4.666667,220.0,20.0,you [Music] [Music] [Music] you I...,you music music music you i...,"[you, music, music, music, you, i, you, the, g...","[music, music, music, greatness, country, lead..."
3,-byuyavcNI4,Donald Trump Speeches & Events,20160617,"Full Speech: Donald Trump Rally in Houston, Te...",15138.0,4.582491,266.0,31.0,we welcome stars and president [Music] ...,we welcome stars and president music ...,"[we, welcome, stars, and, president, music, al...","[welcome, stars, president, music, place, live..."
4,09BXh-AA72M,Donald Trump Speeches & Events,20161105,"Full Speech: Donald Trump Rally in Denver, Col...",8720.0,4.924731,365.0,7.0,you thank you [Music] great people Gr...,you thank you music great people gr...,"[you, thank, you, music, great, people, granna...","[thank, music, great, people, granna, three, d..."


In [148]:
trump_speech_df['playlist'].value_counts()

Donald Trump Rally Speech Events Press Conference Rallies Playlist    369
President Donald Trump's First 100 Days | NBC News                    233
Donald Trump Speeches & Events                                        135
DONALD TRUMP SPEECHES & PRESS CONFERENCE                               77
President Donald Trump Weekly Address 2017                             22
Name: playlist, dtype: int64

In [149]:
trump_speech_df.sort_values('upload_date')

,id,playlist,upload_date,title,view_count,average_rating,like_count,dislike_count,subtitles,subtitle1,tokenized_subtitle,subtitle_stop
795,WPnzNomBjuE,Donald Trump Rally Speech Events Press Confere...,20160201,LIVE Donald Trump Cedar Rapids Iowa Rally Doub...,1728.0,5.000000,38.0,0.0,prism growing government and you know all...,prism growing government and you know all...,"[prism, growing, government, and, you, know, a...","[prism, growing, government, know, allowing, t..."
832,ZqINGT-_Dno,Donald Trump Rally Speech Events Press Confere...,20160201,LIVE Donald Trump Rally in Waterloo Iowa on Ca...,8150.0,4.586207,130.0,15.0,time we've all worked very hard for this ...,time we ve all worked very hard for this ...,"[time, we, ve, all, worked, very, hard, for, t...","[time, worked, hard, wanted, make, stop, going..."
653,KAMonkoVG4o,Donald Trump Rally Speech Events Press Confere...,20160202,LIVE Donald Trump Rally Milford New Hampshire ...,26696.0,4.738095,471.0,33.0,we love you we love you amazing it's amaz...,we love you we love you amazing it s amaz...,"[we, love, you, we, love, you, amazing, it, s,...","[love, love, amazing, amazing, know, one, repo..."
738,QpKhuMbYpm0,Donald Trump Rally Speech Events Press Confere...,20160205,LIVE Donald Trump Florence South Carolina Civi...,24555.0,4.741071,419.0,29.0,I was talking to somebody and then I talk...,i was talking to somebody and then i talk...,"[i, was, talking, to, somebody, and, then, i, ...","[talking, somebody, talked, reporter, good, re..."
597,fjPJ6c7J0tM,Donald Trump Rally Speech Events Press Confere...,20160207,LIVE Donald Trump Plymouth New Hampshire FULL ...,11145.0,4.796078,242.0,13.0,you you Wow this is beautiful we just k...,you you wow this is beautiful we just k...,"[you, you, wow, this, is, beautiful, we, just,...","[wow, beautiful, kept, driving, driving, drivi..."
92,JLZ8e4JYxe4,Donald Trump Speeches & Events,20160219,Full Speech: Donald Trump Rally in Pawley's Is...,1537.0,4.684210,35.0,3.0,you you thank you very much do you know...,you you thank you very much do you know...,"[you, you, thank, you, very, much, do, you, kn...","[thank, much, know, many, people, outside, kno..."
500,5qbCHV2KLLw,Donald Trump Rally Speech Events Press Confere...,20160220,LIVE Donald Trump Winning Speech Caucus South ...,11675.0,4.692307,168.0,14.0,everybody well I want to begin by thankin...,everybody well i want to begin by thankin...,"[everybody, well, i, want, to, begin, by, than...","[everybody, well, want, begin, thanking, peopl..."
0,-2WTNSujhjk,Donald Trump Speeches & Events,20160220,Live Stream: Donald Trump Victory Rally in Spa...,4057.0,4.259259,44.0,10.0,presidents of the United States mr. go tr...,presidents of the united states mr go tr...,"[presidents, of, the, united, states, mr, go, ...","[presidents, united, states, mr, go, trapp, fa..."
732,QAFYfsA28-Q,Donald Trump Rally Speech Events Press Confere...,20160221,LIVE Donald Trump Rally Georgia World Congress...,23641.0,4.791123,363.0,20.0,we love you we love you we had such an am...,we love you we love you we had such an am...,"[we, love, you, we, love, you, we, had, such, ...","[love, love, amazing, victory, yesterday, incr..."
673,lTYUoPvut8I,Donald Trump Rally Speech Events Press Confere...,20160222,LIVE Donald Trump Las Vegas Nevada Rally South...,16176.0,4.607843,276.0,30.0,this is a big beautiful building Wow unbe...,this is a big beautiful building wow unbe...,"[this, is, a, big, beautiful, building, wow, u...","[big, beautiful, building, wow, unbelievable, ..."


In [150]:
lemmatizer = WordNetLemmatizer()
trump_speech_df['subtitle_lemma']=trump_speech_df['subtitle_stop'].apply(lambda x : [lemmatizer.lemmatize(y) for y in x])

In [151]:
def func(row):
    return " ".join(row)
trump_speech_df['data'] = trump_speech_df['subtitle_lemma'].apply(lambda x: func(x))
trump_speech_df.head()

,id,playlist,upload_date,title,view_count,average_rating,like_count,dislike_count,subtitles,subtitle1,tokenized_subtitle,subtitle_stop,subtitle_lemma,data
0,-2WTNSujhjk,Donald Trump Speeches & Events,20160220,Live Stream: Donald Trump Victory Rally in Spa...,4057.0,4.259259,44.0,10.0,presidents of the United States mr. go tr...,presidents of the united states mr go tr...,"[presidents, of, the, united, states, mr, go, ...","[presidents, united, states, mr, go, trapp, fa...","[president, united, state, mr, go, trapp, fami...",president united state mr go trapp family than...
1,-64nfy6i58w,Donald Trump Speeches & Events,20161107,LAST RALLY: Donald Trump FINAL CAMPAIGN Rally ...,47276.0,4.358025,952.0,182.0,it's now officially Tuesday November a di...,it s now officially tuesday november a di...,"[it, s, now, officially, tuesday, november, a,...","[officially, tuesday, november, ever, think, h...","[officially, tuesday, november, ever, think, h...",officially tuesday november ever think hearing...
2,-7Sp31hTxkU,Donald Trump Speeches & Events,20160423,"FULL SPEECH: Donald Trump Rally in Bridgeport,...",19966.0,4.666667,220.0,20.0,you [Music] [Music] [Music] you I...,you music music music you i...,"[you, music, music, music, you, i, you, the, g...","[music, music, music, greatness, country, lead...","[music, music, music, greatness, country, lead...",music music music greatness country leader fre...
3,-byuyavcNI4,Donald Trump Speeches & Events,20160617,"Full Speech: Donald Trump Rally in Houston, Te...",15138.0,4.582491,266.0,31.0,we welcome stars and president [Music] ...,we welcome stars and president music ...,"[we, welcome, stars, and, president, music, al...","[welcome, stars, president, music, place, live...","[welcome, star, president, music, place, live,...",welcome star president music place live place ...
4,09BXh-AA72M,Donald Trump Speeches & Events,20161105,"Full Speech: Donald Trump Rally in Denver, Col...",8720.0,4.924731,365.0,7.0,you thank you [Music] great people Gr...,you thank you music great people gr...,"[you, thank, you, music, great, people, granna...","[thank, music, great, people, granna, three, d...","[thank, music, great, people, granna, three, d...",thank music great people granna three day goin...


In [152]:
tf_vectorizer = TfidfVectorizer(min_df=5,max_df=0.75,ngram_range=(2,3),max_features=7000) 
# min_df - minimum number of documents a word should appear in. 
# max_df - max percentage of documents a word should appear in. Here 80%. If it is more than this, word is probably redundant
train_data_features_TF = tf_vectorizer.fit_transform(trump_speech_df['data'])
idf= tf_vectorizer.idf_
train_data_features_TF=train_data_features_TF.toarray()
d=dict(zip(tf_vectorizer.get_feature_names(), idf))
df=pd.DataFrame(list(d.items()),columns=['Words','Scores'])
df=df.sort_values('Scores')
df.tail()

,Words,Scores
6242,trump clinton,5.938065
5339,senator rubio,5.938065
831,closed plant,5.938065
21,acting attorney general,5.938065
20,acting attorney,5.938065


In [153]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()
# print_top_words(lda,count_feature_names,10)

In [154]:
print("Extracting tf features for LDA...")
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=5,
                                max_features=7000,
                                stop_words='english')

tf = tf_vectorizer.fit_transform(trump_speech_df['data'])

lda = LatentDirichletAllocation(n_topics=25, max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)

lda.fit(tf)

Extracting tf features for LDA...


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=50.0,
             max_doc_update_iter=100, max_iter=5, mean_change_tol=0.001,
             n_jobs=1, n_topics=25, perp_tol=0.1, random_state=0,
             topic_word_prior=None, total_samples=1000000.0, verbose=0)

In [155]:
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, 20)

Topic #0: going people know country want great job right state said time like got way american thing hillary think make clinton
Topic #1: going country people job great want make american applause know right thank like america veteran new time care said president
Topic #2: going people know like country right want said job great time thank say look hillary think thing trump make president
Topic #3: people going know country want said great say right make like job think thing way american hillary really got look
Topic #4: going people know country want right like say great said job think year thing way lot look trump state tell
Topic #5: going people country american know job make new want look america said like right clinton time say think great thank
Topic #6: going people know country want said job think like great say hillary right dollar make money american time state clinton
Topic #7: going know people right country like great said say make want look applause job lot time thank wa

In [156]:
inaugural_day = trump_speech_df.loc[trump_speech_df['upload_date']==20170120]

In [157]:
inaugural_day.shape

(17, 14)

In [158]:
bins = [20160201, 20160720, 20161108, 20170119, 20170120,20170803]
trump_speech_df['binned'] = pd.cut(trump_speech_df['upload_date'], bins,labels=['RNC-campaign','Presidential Campaign','President-Elected','Inaugural-Day','President'])
trump_speech_df['binned'].unique()

[RNC-campaign, Presidential Campaign, President-Elected, Inaugural-Day, President, NaN]
Categories (5, object): [RNC-campaign < Presidential Campaign < President-Elected < Inaugural-Day < President]

In [112]:
trump_speech_df.head()

,id,playlist,upload_date,title,view_count,average_rating,like_count,dislike_count,subtitles,subtitle1,tokenized_subtitle,subtitle_stop,subtitle_lemma,data,binned
0,-2WTNSujhjk,Donald Trump Speeches & Events,20160220,Live Stream: Donald Trump Victory Rally in Spa...,4057.0,4.259259,44.0,10.0,presidents of the United States mr. go tr...,presidents of the united states mr go tr...,"[presidents, of, the, united, states, mr, go, ...","[presidents, united, states, mr, go, trapp, fa...","[president, united, state, mr, go, trapp, fami...",president united state mr go trapp family than...,RNC-campaign
1,-64nfy6i58w,Donald Trump Speeches & Events,20161107,LAST RALLY: Donald Trump FINAL CAMPAIGN Rally ...,47276.0,4.358025,952.0,182.0,it's now officially Tuesday November a di...,it s now officially tuesday november a di...,"[it, s, now, officially, tuesday, november, a,...","[officially, tuesday, november, ever, think, h...","[officially, tuesday, november, ever, think, h...",officially tuesday november ever think hearing...,Presidential Campaign
2,-7Sp31hTxkU,Donald Trump Speeches & Events,20160423,"FULL SPEECH: Donald Trump Rally in Bridgeport,...",19966.0,4.666667,220.0,20.0,you [Music] [Music] [Music] you I...,you music music music you i...,"[you, music, music, music, you, i, you, the, g...","[music, music, music, greatness, country, lead...","[music, music, music, greatness, country, lead...",music music music greatness country leader fre...,RNC-campaign
3,-byuyavcNI4,Donald Trump Speeches & Events,20160617,"Full Speech: Donald Trump Rally in Houston, Te...",15138.0,4.582491,266.0,31.0,we welcome stars and president [Music] ...,we welcome stars and president music ...,"[we, welcome, stars, and, president, music, al...","[welcome, stars, president, music, place, live...","[welcome, star, president, music, place, live,...",welcome star president music place live place ...,RNC-campaign
4,09BXh-AA72M,Donald Trump Speeches & Events,20161105,"Full Speech: Donald Trump Rally in Denver, Col...",8720.0,4.924731,365.0,7.0,you thank you [Music] great people Gr...,you thank you music great people gr...,"[you, thank, you, music, great, people, granna...","[thank, music, great, people, granna, three, d...","[thank, music, great, people, granna, three, d...",thank music great people granna three day goin...,Presidential Campaign


In [110]:
rnc_df = trump_speech_df.loc[trump_speech_df['binned'] == 'RNC-campaign']
rnc_df.shape

(168, 15)

In [111]:
pc_df = trump_speech_df.loc[trump_speech_df['binned'] == 'Presidential Campaign']
pc_df.shape

(262, 15)

In [113]:
elected_df = trump_speech_df.loc[trump_speech_df['binned'] == 'President-Elected']
elected_df.shape

(13, 15)

In [114]:
inaugural_df = trump_speech_df.loc[trump_speech_df['binned'] == 'Inaugural-Day']
inaugural_df.shape

(17, 15)

In [115]:
president_df = trump_speech_df.loc[trump_speech_df['binned'] == 'President']
president_df.shape

(374, 15)

In [120]:
# RNC Campaign
tf_vectorizer = CountVectorizer(max_df=0.7, min_df=5,
                                max_features=1000,
                                stop_words='english')

tf = tf_vectorizer.fit_transform(rnc_df['data'])

lda = LatentDirichletAllocation(n_topics=25, max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)

lda.fit(tf)
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, 20)

Topic #0: worker china mike clinton plant business nafta mexico plan company send tax closed protect product hampshire tpp manufacturing moved massive
Topic #1: china mexico american clinton oh went story veteran business build send change car getting tax billion somebody probably poll bernie
Topic #2: applause indiana clinton penny billion business tax company american horrible build mike person pay obama music mexico leader probably week
Topic #3: clinton china amazing mexico politician story pay company american debate went obama ago billion isi tremendous talking build business single
Topic #4: god mexico pledge stand bless run nation oh family billion incredible strong justice liberty stuff amazing went winning applause american
Topic #5: oh politician china understand ago incredible smart billion talking mexico went wanted story getting business company beautiful drug making unbelievable
Topic #6: american china clinton business carolina mexico veteran law north understand away n

In [119]:
# Presidential Campaign
tf_vectorizer = CountVectorizer(max_df=0.7, min_df=5,
                                max_features=1000,
                                stop_words='english')

tf = tf_vectorizer.fit_transform(pc_df['data'])

lda = LatentDirichletAllocation(n_topics=25, max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)

lda.fit(tf)
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, 20)

Topic #0: african school city applause community child story education virginia inner campaign seen student vote medium tax bad future billion happen
Topic #1: remember applause win border vote na trade okay gon folk tax deal mexico bad music plan wall thousand happen obamacare
Topic #2: tax poland bad seen deal important happen thousand million community understand story applause vote special trade movement york regulation totally
Topic #3: tax win applause remember vote folk na thousand illegal number oh city community law company deal trade okay gon end
Topic #4: tax deal million trade applause win happen okay thousand gon border music oh happened folk wall child pay plan city
Topic #5: mexico border illegal tax trade drug number city thousand problem happen deal win applause respect tremendous million working vote oh
Topic #6: mr general honor security military support million national bless hotel special fact god campaign commander deal leadership service word family
Topic #7: tax

In [124]:
# Elected
tf_vectorizer = CountVectorizer(max_df=0.7, min_df=2,
                                max_features=200,
                                stop_words='english')

tf = tf_vectorizer.fit_transform(elected_df['data'])

lda = LatentDirichletAllocation(n_topics=10, max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)

lda.fit(tf)
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, 20)

Topic #0: deal russia organization news office mexico question trust hacking conflict intelligence meeting report government look asset mr industry putin fake
Topic #1: indiana elected greatest industry unbelievable life heart night disgrace health lost set close horrible meeting effort line office single general
Topic #2: carolina north military defense protect longer win men seen billion better flag reform act point trade general woman veteran create
Topic #3: deal look government organization win russia mexico trust started ohio woman mr trade military news point meeting taking intelligence carolina
Topic #4: woman vote trade look forgotten thanks deal close community iowa win utah ago market came point men election tremendous veteran
Topic #5: thanks mexico deal trust organization russia news asset government released michael fake single word office veteran question action happening court
Topic #6: technology plant mexico indiana called greg woman everybody north leaving defense de

In [123]:
# Inaugural Day
tf_vectorizer = CountVectorizer(max_df=0.7, min_df=2,
                                max_features=200,
                                stop_words='english')

tf = tf_vectorizer.fit_transform(inaugural_df['data'])

lda = LatentDirichletAllocation(n_topics=10, max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)

lda.fit(tf)
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, 20)

Topic #0: office border million talking life step speak allegiance hear ability general great bring lady strength best build moment capital american
Topic #1: god country john want job america nation long challenge decision life different destiny united government movement dream city citizen industry
Topic #2: going thank like music people great know think make want world everybody tell good right general lot wanted campaign state
Topic #3: know penny time help united state got look god office nbc swear solemnly defend right constitution fan news michael vice
Topic #4: america american country united applause people nation state trump god obama right new world donald great dream make today job
Topic #5: american america people country world stop god today wealth michelle life nation men bring power talk industry victory got lead
Topic #6: going music great america world american people build everybody thank think campaign elect power woman change day country washington man
Topic #7: go

In [126]:
# President
tf_vectorizer = CountVectorizer(max_df=0.7, min_df=3,
                                max_features=300,
                                stop_words='english')

tf = tf_vectorizer.fit_transform(president_df['data'])

lda = LatentDirichletAllocation(n_topics=25, max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)

lda.fit(tf)
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, 20)

Topic #0: thank united state country american world judge america great today day know nation god mr make want year like music
Topic #1: nation state american new peace israel united world thank terrorism country leader time going think security america way great year
Topic #2: united state country minister going prime thank trade good trump work think mr agreement great security know job continue world
Topic #3: think democrat obamacare health care really going hard vote thing happen know thank great support year lot better good party
Topic #4: meeting united policy american state new thank world country think going question agreement america say great security different terrorism secretary
Topic #5: going think united right american said country lot know talk saying way thing like work good agreement trade state fair
Topic #6: house white trump know think intelligence report say said question press donald russian right story fact time news obama campaign
Topic #7: today world great s

In [145]:
trump_speech_df['binned']

KeyError: 'binned'